For Mr. Rudra

This is the Lending club credit risk analysis project.

# First step: download the dataset

In [304]:
# Download the dataset which includes approved loans data from 2007 to 2011.
# https://www.lendingclub.com/info/download-data.action.
# You can find the meaning of each column of the table by checking DATA DICTIONARY.
# DATA DICTIONARY: https://www.lendingclub.com/info/download-data.action.

import pandas as pd
loans_2007 = pd.read_csv('/Users/liaoyantai/Downloads/LoanStats3a.csv', skiprows=1,\
                         low_memory=False)
print(loans_2007.shape)

(42538, 111)


# Second step: Data cleaning and preparing the features 

In [305]:
# The 'id' column and 'member_id' column are empty, so delete them firstly.

loans = loans_2007.drop(['id', 'member_id'],axis=1)
print(loans.shape)

(42538, 109)


In [306]:
# The 'desc' cloumn contains a long text explanation for each loan. 
# The 'url' cloumn contains a link to each loan on Lending Club which can only be accessed with
# an investor account.
# Drop these two columns.

loans = loans.drop(['desc', 'url'],axis=1)
print(loans.shape)

(42538, 107)


In [307]:
# For efficiency removing all columns containing more than 50% missing values. This may lose 
# some information but speed processing since we can spend less time trying to fill these values.

half_count = len(loans) / 2
loans = loans.dropna(thresh=half_count, axis=1)
print(loans.shape)

(42538, 50)


In [308]:
# Export the table for further reference.

loans.to_csv('/Users/liaoyantai/Downloads/Loans.csv')

In [309]:
# funded_amnt: leaks data from the future (after the loan is already started to be funded).
# funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
# grade: contains redundant information as the interest rate column (int_rate).
# sub_grade: also contains redundant information as the interest rate column (int_rate).
# emp_title: requires other data and a lot of processing to potentially be useful
# issue_d: leaks data from the future (after the loan is already completed funded)

loans=loans.drop(['funded_amnt','funded_amnt_inv',\
                'grade','sub_grade','emp_title','issue_d'],axis=1)
print(loans.shape)


(42538, 44)


In [310]:
# zip_code: redundant with the addr_state column
# out_prncp: leaks data from the future, (after the loan already started to be paid off)
# out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
# total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
# total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
# total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

loans = loans.drop(['zip_code','out_prncp','out_prncp_inv',\
                            'total_pymnt','total_pymnt_inv','total_rec_prncp'],axis=1)
print(loans.shape)

(42538, 38)


In [311]:
# total_rec_int: leaks data from the future, (after the loan already started to be paid off)
# total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off)
# recoveries: also leaks data from the future, (after the loan already started to be paid off)
# collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off)
# last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off)
# last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off)

loans = loans.drop(['total_rec_int','total_rec_late_fee','recoveries',\
                            'collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'],axis=1)
print(loans.shape)

(42538, 32)


In [312]:
# The 'loan_status' column is the target column which we need to predict in the future, so we 
# need to find the all of the unique value of it.

print(loans['loan_status'].value_counts())

Fully Paid                                             34115
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                         1
Name: loan_status, dtype: int64


In [313]:
# The 'loan_sataus' has five different values but we just choose 'Fully Paid' and 'Charged Off',
# and transfer them to 1 and o as binary classification.
# Fully Paid: Loan has been fully paid off.
# Charged off: Loan for which there is no longer a reasonable expectation of further payments.

loans =loans[(loans['loan_status']=='Fully Paid') | (loans['loan_status']=='Charged Off')]
loan_status_mapping_dict = {'loan_status':{'Fully Paid':1,'Charged Off':0 }}
loans =loans.replace(loan_status_mapping_dict)

In [314]:
# Find the columns that contain only one unique value. These columns won't be useful for the 
# model since they don't add any information to each loan application.

columns = loans.columns
drop_columns =[]
for col in columns:
    leng_loans_col=loans[col].dropna().unique()
    if len(leng_loans_col)==1:
        drop_columns.append(col)
        
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [315]:
# Drop these one-value columns.

loans=loans.drop(drop_columns, axis=1)
print(loans.shape)

(39785, 23)


In [316]:
# Return the number of missing values across the Dataframe.

null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


In [317]:
# Drop the 'pub_rec_bankruptcies' column since more than 1% of the rows have a missing value 
# for this column.
# Remove the remaining rows containing null values since these rows' number are less than 1% of 
# total rows.
# For the missing value we also can fill it by the  median value of the correspinding attribute
# from the 10 or 20 closest point. But for this project we don't do this for efficiency. 

loans=loans.drop('pub_rec_bankruptcies',axis=1)
loans=loans.dropna(axis=0)
print(loans.shape)

(39723, 22)


In [318]:
# Find the data type for all columns since we can only fit model by numerical variables.

print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


In [319]:
# Find all non numerical columns.

object_columns_df = loans.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Mar-2017
Name: 0, dtype: object


In [320]:
# Find the unique value of column of 'purpose' and column of 'title'. The result shows that they
# are almost same, so we will drop 'title' columns since it is more complicated.

print(loans['purpose'].value_counts())

print(loans['title'].value_counts())

debt_consolidation    18660
credit_card            5134
other                  3985
home_improvement       2980
major_purchase         2182
small_business         1827
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                                               2188
Debt Consolidation Loan                                          1732
Personal Loan                                                     661
Consolidation                                                     516
debt consolidation                                                508
Credit Card Consolidation                                         357
Home Improvement                                                  357
Debt consolidation                                                334
Small Business Loan 

In [321]:
# title: loan title provided the borrower
# last_credit_pull_d: The most recent month Lending Club pulled credit for this loan
# addr_state: borrower's state of residence
# earliest_cr_line: The month the borrower's earliest reported credit line was opened

loans=loans.drop(['title','last_credit_pull_d','addr_state','earliest_cr_line'],axis=1)
print(loans.shape)

(39723, 18)


In [322]:
# Find the unique value of 'home_ownership', 'verification_status', 'emp_length' and'term'.

cols = ['home_ownership', 'verification_status', 'emp_length', 'term']

for col in cols:
    print(loans[col].value_counts())

RENT        18881
MORTGAGE    17687
OWN          3056
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16890
Verified           12832
Source Verified    10001
Name: verification_status, dtype: int64
10+ years    8897
< 1 year     4576
2 years      4389
3 years      4094
4 years      3435
5 years      3279
1 year       3240
6 years      2227
7 years      1771
8 years      1482
9 years      1259
n/a          1074
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10682
Name: term, dtype: int64


In [323]:
# Transfer column of 'emp_length' to numerical type.

emp_length_mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}


loans=loans.replace(emp_length_mapping_dict)


In [324]:
# Transfer these columns to numerical type.

loans['int_rate']=(loans['int_rate'].str.rstrip('%').astype('float'))
loans['revol_util']=loans['revol_util'].str.rstrip('%').astype('float')

In [325]:
# Encode the 'home_ownership', 'verification_status', 'title', and 'term' columns as dummy 
# variables so we can use them in our model. 

cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)
print(loans.shape)

(39723, 38)


In [326]:
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39723 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              39723 non-null float64
int_rate                               39723 non-null float64
installment                            39723 non-null float64
emp_length                             39723 non-null int64
annual_inc                             39723 non-null float64
loan_status                            39723 non-null int64
dti                                    39723 non-null float64
delinq_2yrs                            39723 non-null float64
inq_last_6mths                         39723 non-null float64
open_acc                               39723 non-null float64
pub_rec                                39723 non-null float64
revol_bal                              39723 non-null float64
revol_util                             39723 non-null float64
total_acc                              39723 non-null float64
home_ownership_MORTGAGE    

In [327]:
# Export the final table

loans.to_csv('/Users/liaoyantai/Downloads/Loans_final.csv')

In [328]:
# Create independent variables and dependent variable for fitting model.

features = loans.drop('loan_status',axis=1)
target = pd.Series(loans['loan_status'])

In [329]:
# There are 6 times as many loans that were paid off on time (1), than loans that weren't paid 
# off on time (0). There is a significant class imbalance in the loan_status column.

print(len(target[target==1]))
print(len(target[target==0]))

34072
5651


# Third step: Making predictions

  Use logistic regression and random forest algorithms to fit the model and make the      predicitons.

  In order to get a realistic depiction of the accuracy of the algorithm and avoid the overfit, I use 10 folds cross validation to generate predictions.  

  An error metric will help us figure out when our model is performing well, and when it's performing poorly. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. In this case, we're primarily concerned with false positives and false negatives. 

  With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

  Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

  Tell the classifier to penalize misclassifications of the less prevalent class more than the other class to get a classifier to correct for imbalanced classes.

In [330]:
# set class_weight equal to 'balanced' to penalize the misclassifications in classifer.

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold

lr = LogisticRegression(class_weight='balanced')
kf = KFold(loans.shape[0], n_folds =10,random_state=1)

predictions = cross_val_predict(lr, features, target, cv=kf)
loans['predictions']=predictions

tn_filter = (loans['predictions']==0)&(loans['loan_status']==0)
tn = len(loans[tn_filter])

tp_filter = (loans['predictions']==1)&(loans['loan_status']==1)
tp = len(loans[tp_filter])

fn_filter = (loans['predictions']==0)&(loans['loan_status']==1)
fn = len(loans[fn_filter])

fp_filter = (loans['predictions']==1)&(loans['loan_status']==0)
fp = len(loans[fp_filter])

fpr = fp/(fp+tn)
print(fpr)

tpr = tp/(tp+fn)
print(tpr)

0.3739161210405238
0.6499765203099319


In [331]:
# set class_weight equal to dict of penalty to penalize the misclassifications in classifer.

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict

penalty = {
    0:10,
    1:1
}

lr=LogisticRegression(class_weight= penalty)
kf = KFold(loans.shape[0], n_folds =10,random_state=1)

predictions = cross_val_predict(lr, features, target, cv=kf)
loans['predictions']=predictions

tn_filter = (loans['predictions']==0)&(loans['loan_status']==0)
tn = len(loans[tn_filter])

tp_filter = (loans['predictions']==1)&(loans['loan_status']==1)
tp = len(loans[tp_filter])

fn_filter = (loans['predictions']==0)&(loans['loan_status']==1)
fn = len(loans[fn_filter])

fp_filter = (loans['predictions']==1)&(loans['loan_status']==0)
fp = len(loans[fp_filter])

fpr = fp/(fp+tn)
print(fpr)

tpr = tp/(tp+fn)
print(tpr)

0.08405591930631746
0.2343273068795492


In [332]:
# set class_weight equal to None.

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict


rf=RandomForestClassifier(random_state =1)
kf = KFold(features.shape[0],n_folds=10, random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)

loans['predictions']=predictions

tn_filter = (loans['predictions']==0)&(loans['loan_status']==0)
tn = len(loans[tn_filter])

tp_filter = (loans['predictions']==1)&(loans['loan_status']==1)
tp = len(loans[tp_filter])

fn_filter = (loans['predictions']==0)&(loans['loan_status']==1)
fn = len(loans[fn_filter])

fp_filter = (loans['predictions']==1)&(loans['loan_status']==0)
fp = len(loans[fp_filter])

fpr = fp/(fp+tn)
print(fpr)

tpr = tp/(tp+fn)
print(tpr)

0.914528402052734
0.9653381075369805


In [333]:
# set class_weight equal to 'balanced' to penalize the misclassifications in classifer.

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict

rf=RandomForestClassifier(random_state =1,class_weight= 'balanced')
kf = KFold(features.shape[0],n_folds=10, random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)

loans['predictions']=predictions

tn_filter = (loans['predictions']==0)&(loans['loan_status']==0)
tn = len(loans[tn_filter])

tp_filter = (loans['predictions']==1)&(loans['loan_status']==1)
tp = len(loans[tp_filter])

fn_filter = (loans['predictions']==0)&(loans['loan_status']==1)
fn = len(loans[fn_filter])

fp_filter = (loans['predictions']==1)&(loans['loan_status']==0)
fp = len(loans[fp_filter])

fpr = fp/(fp+tn)
print(fpr)

tpr = tp/(tp+fn)
print(tpr)

0.9451424526632455
0.975786569617281


In [334]:
# set class_weight equal to dict of penalty to penalize the misclassifications in classifer.

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict

penalty = {
    0:10,
    1:1
}

rf=RandomForestClassifier(random_state =1,class_weight= penalty)
kf = KFold(features.shape[0],n_folds=10, random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)

loans['predictions']=predictions

tn_filter = (loans['predictions']==0)&(loans['loan_status']==0)
tn = len(loans[tn_filter])

tp_filter = (loans['predictions']==1)&(loans['loan_status']==1)
tp = len(loans[tp_filter])

fn_filter = (loans['predictions']==0)&(loans['loan_status']==1)
fn = len(loans[fn_filter])

fp_filter = (loans['predictions']==1)&(loans['loan_status']==0)
fp = len(loans[fp_filter])

fpr = fp/(fp+tn)
print(fpr)

tpr = tp/(tp+fn)
print(tpr)

0.9451424526632455
0.9753756750410895


# Fourth step: Conclusion

1, Logistic regression (class_weight='balanced'):
   fpr =  0.3739161210405238
   tpr =  0.6499765203099319
   
2, Logistic regression (class_weight= dict of penalty):
   fpr =  0.08405591930631746
   tpr =  0.2343273068795492
   
3, RandomForest regression (class_weight= None):
   fpr =  0.914528402052734
   tpr =  0.9653381075369805
   
4, RandomForest regression (class_weight= 'balanced'):
   fpr =  0.9451424526632455
   tpr =  0.975786569617281
   
5, RandomForest regression (class_weight= dict of penalty):
   fpr =  0.9451424526632455
   tpr =  0.9753756750410895
   
From the above result we can see that the second model has the lowest fpr (the lowest risk). Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs. 

Of course we may improve our models by trying more different penalties or applying different algorithm (even the ensemble of the different algorithms). We also can pay more attention to  the attributes selection and preparation, or derivating the new attributes by combinating the different attributes. These steps may improve our model performance.
   